In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import geopandas as gpd
import warnings
import lightgbm as lgb
import optuna
from sklearn.neighbors import BallTree
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_percentage_error

warnings.filterwarnings('ignore')

# Display all columns
pd.set_option('display.max_columns', None)
input_path = '../input/'
gdf_land = gpd.read_file(os.path.join(input_path, 'L02-25.geojson'))
df_land = gdf_land.copy()
df_land['price'] = df_land['L02_006']
df = pd.read_csv(os.path.join(input_path, 'train.csv'))
test = pd.read_csv(os.path.join(input_path, 'test.csv'))
gdf_land.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 21431 entries, 0 to 21430
Columns: 140 entries, L02_001 to geometry
dtypes: float64(4), geometry(1), int32(69), object(66)
memory usage: 17.2+ MB


In [16]:
# データ前処理関数．特徴量作成もここで
def preprocess_data(input_df, df_land, is_train=True, fill_value = None):
    """
    駅情報は一旦無視！
    「巨大なゴミデータ」の削除と、「地価・築年数」だけに集中したシンプル版
    """
    df = input_df.copy()
    
    # -----------------------------------------------------
    # 0. 【Trainのみ】ハズレ値などの削除
    # -----------------------------------------------------
    if is_train:
        # 1. 面積フィルタ
        # 面積500㎡以上はカット
        df = df[df['unit_area'] < 500] 
        
        # 2. 単価がおかしい土地を削除
        # 単価(円/㎡)を計算
        temp_unit_price = df['money_room'] / df['unit_area']
        
        # 「単価が1万円/㎡ 以下」は、原野や山林なので捨てる
        df = df[temp_unit_price > 10000] 
        
        # 3. 「高すぎる単価」もノイズになるのでキャップを設ける
        df = df[temp_unit_price < 10000000]

    else:
        # Testデータの場合、面積の欠損値を中央値で埋める
        fill_val = fill_value if fill_value else df['unit_area'].median()
        if fill_value is not None:
            df['unit_area'] = df['unit_area'].fillna(fill_value)
    # -----------------------------------------------------
    # 1. 駅情報 (今後実装予定)
    # -----------------------------------------------------
    # ※カラム名が画像通り (bus_time1, walk_distance1) 前提で書きます
    # もし違ったら df.columns で確認して書き換えてください！
    
    # 必要な列があるか確認
    if 'walk_distance1' in df.columns:
        
        def calc_access_time(row, suffix):
            """
            suffix: '1' または '2' を指定
            """
            # 1. データの取得 (欠損値は0にしておく)
            w_dist = row.get(f'walk_distance{suffix}', np.nan)
            b_time = row.get(f'bus_time{suffix}', np.nan)
            
            # データが全くない場合は「不明(無限大)」
            if pd.isna(w_dist) and pd.isna(b_time):
                return 999.0 # あとでminを取るときに負けるように大きくする

            # NaNを0扱いで計算用に確保
            w_dist_val = 0 if pd.isna(w_dist) else w_dist
            b_time_val = 0 if pd.isna(b_time) else b_time
            
            # 2. 徒歩時間を計算 (80m = 1分
            walk_min = w_dist_val / 80.0
            
            # 3. ロジック分岐
            if b_time_val > 0:
                # 【パターンA: バス利用】
                # 時間 = バス乗車時間 + バス停からの徒歩 + 待ち時間ペナルティ(10分)
                return b_time_val + walk_min + 10.0
            else:
                # 【パターンB: 徒歩のみ】
                # 時間 = 駅からの徒歩
                return walk_min

        # ルート1とルート2、それぞれの所要時間を計算
        df['access_time_1'] = df.apply(lambda x: calc_access_time(x, '1'), axis=1)
        df['access_time_2'] = df.apply(lambda x: calc_access_time(x, '2'), axis=1)
        
        # 「1」と「2」のうち、時間が短い方（＝近い方）を採用！
        # どっちも999(不明)なら、最終的に120分で埋める
        df['station_minutes'] = df[['access_time_1', 'access_time_2']].min(axis=1)
        
        # 最終的な欠損値埋め (田舎対策)
        # 999のまま残ったやつは「駅徒歩圏外」として120分にする
        df['station_minutes'] = df['station_minutes'].replace(999.0, 120.0)
        
        # 念のため station_minutes 自体がNaNのやつも埋める
        df['station_minutes'] = df['station_minutes'].fillna(120.0)

        # 不要になった一時的な列を削除（メモリ節約）
        df.drop(['access_time_1', 'access_time_2'], axis=1, inplace=True)
        
        # カテゴリ用 (沿線名などは _1 の方を正として採用しておく)
        if 'rosen_name1' in df.columns:
            df['line_name'] = df['rosen_name1']
            df['station_name'] = df['eki_name1']
        else:
            df['line_name'] = 'unknown'
            df['station_name'] = 'unknown'

    else:
        # そもそも列がない場合
        df['station_minutes'] = 120.0
        df['line_name'] = 'unknown'
        df['station_name'] = 'unknown'

    # -----------------------------------------------------
    # 2. 築年数の計算 
    # -----------------------------------------------------
    df['year_built'] = pd.to_numeric(df['year_built'], errors='coerce')
    df['year_built'] = df['year_built'].fillna(df['year_built'].median())
    df['temp_time'] = pd.to_datetime(df['year_built'].astype(int).astype(str), format='%Y%m', errors='coerce')
    
    base_date = pd.to_datetime('2025-12-01')
    df['building_month'] = (base_date - df['temp_time']).dt.days / 30.44
    df['building_month'] = df['building_month'].fillna(df['building_month'].median())
    df['building_month'] = df['building_month'].astype(float)

    # -----------------------------------------------------
    # 3. 地価データの結合 
    # -----------------------------------------------------
    if not df_land.empty:
        df_land['lat'] = df_land.geometry.y
        df_land['lon'] = df_land.geometry.x
        land_clean = df_land[['lat', 'lon', 'price']].dropna()
        
        # データがある場合のみBallTree
        if len(land_clean) > 0:
            land_rad = np.deg2rad(land_clean[['lat', 'lon']])
            input_rad = np.deg2rad(df[['lat', 'lon']])
            
            tree = BallTree(land_rad, metric='haversine')
            dists, indices = tree.query(input_rad, k=1)
            
            df['land_price'] = land_clean['price'].values[indices.flatten()]
            df['dist_to_land_price'] = dists.flatten() * 6371 * 1000
            df['log_land_price'] = np.log1p(df['land_price'])
        else:
             df['log_land_price'] = 0
             df['dist_to_land_price'] = 0
    else:
        df['log_land_price'] = 0
        df['dist_to_land_price'] = 0

    # 4. 地価フィルタ (Trainのみ)
    # 正しい地価データが入っていれば有効化
    if is_train and 'log_land_price' in df.columns:
         df = df[df['log_land_price'] > 5] 

    # -----------------------------------------------------
    # 5. 特徴量リスト
    # -----------------------------------------------------
    cat_cols = [
        'addr1_2', 
        'rosen_name1',
        'eki_name1',
        'building_structure',
        'snapshot_window_direction',
        'madori_kind_all'
    ]
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).fillna('unknown').astype('category')
            
    # 数値特徴量からも station_minutes を削除
    num_features = ['unit_area', 'log_land_price', 'dist_to_land_price', 'building_month', 'station_minutes']
    features = num_features + [c for c in cat_cols if c in df.columns]

    target = None
    if is_train:
        # 単価計算
        unit_price = df['money_room'] / df['unit_area']
        target = np.log1p(unit_price)

    return df, features, target

In [17]:
# 設定
class Config:
    n_folds = 5
    seed = 42

    run_optuna_lgb = False
    run_optuna_cb  = False

    lgb_params = {
        'objective': 'regression',
        'metric': 'mape',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'seed': 42,
        'num_leaves': 64,
        'max_depth': -1,
        'min_child_samples': 20,
        'colsample_bytree': 0.8,
        'subsample': 0.8,
        'learning_rate': 0.1717617686033552, 
        'depth': 10, 
        'l2_leaf_reg': 2.3990529382611974e-08
    }

    cb_params = {
        'loss_function': 'RMSE',
        'learning_rate':  0.08642976242905503,
        'iterations': 10000,
        'depth': 10,
        'random_seed': 42,
        'verbose': 0, # ログを出さない
        'early_stopping_rounds': 100,
        'task_type': 'GPU',
        'l2_leaf_reg': 0.0010917664921743996
    }

# 汎用トレーナー関数

def train_model(model_name, X, y, test_df, params, features, cat_features):
    print(f"🚀 {model_name} Training Started")

    kf = KFold(n_splits = Config.n_folds, shuffle = True, random_state = Config.seed)

    oof_preds = np.zeros(len(X))
    test_preds_sum = np.zeros(len(test_df))
    cv_scores = []

    for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
        X_train, X_val = X.iloc[train_index][features], X.iloc[val_index][features]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        if model_name == 'LightGBM':
            lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=cat_features)
            lgb_val = lgb.Dataset(X_val, y_val, reference = lgb_train,categorical_feature=cat_features)
            model = lgb.train(
                params, 
                lgb_train, 
                valid_sets = [lgb_train, lgb_val],
                num_boost_round = 10000,
                callbacks = [lgb.early_stopping(100, verbose=False)]
            )
            val_pred = model.predict(X_val)
            test_pred = model.predict(test_df[features])
        
        elif model_name == 'CatBoost':
            model = CatBoostRegressor(**params)
            model.fit(
                X_train, y_train,
                eval_set=(X_val, y_val),
                cat_features=cat_features,
                use_best_model=True,
                verbose=False
            )
            val_pred = model.predict(X_val)
            test_pred = model.predict(test_df[features])
        
        #結果格納
        oof_preds[val_index] = val_pred
        test_preds_sum += test_pred

        #スコア計算(MAPE)
        score = mean_absolute_percentage_error(np.expm1(y_val), np.expm1(val_pred)) * 100
        cv_scores.append(score)
        print(f"  Fold {fold + 1} MAPE: {score:.4f}%")

    avg_score = np.mean(cv_scores)
    print(f"🚀 {model_name} Training Completed. Average MAPE: {avg_score:.4f}%\n")

    return oof_preds, test_preds_sum / Config.n_folds, avg_score

In [18]:
# Optunaラッパー関数

def run_optuna(model_name, X, y, features, cat_features):
    print(f"🚀 {model_name} Optuna Hyperparameter Tuning Started")

    def objective(trial):
        if model_name == 'lgbm':
            params = {
                'objective': 'regression', 'metric': 'rmse', 'verbosity': -1, 'boosting_type': 'gbdt',
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 32, 256),
                'max_depth': trial.suggest_int('max_depth', 5, 12),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            }
        else: # catboost
            params = {
                'loss_function': 'RMSE', 'verbose': 0, 'early_stopping_rounds': 50, 'iterations': 1000, # 探索時は少なめに
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'depth': trial.suggest_int('depth', 4, 10),
                'task_type': 'GPU',
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
            }
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        scores = []
        for tr_idx, val_idx in kf.split(X, y):
            X_tr, X_val = X.iloc[tr_idx][features], X.iloc[val_idx][features]
            y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
            
            if model_name == 'lgbm':
                dtrain = lgb.Dataset(X_tr, y_tr, categorical_feature=cat_features)
                dval = lgb.Dataset(X_val, y_val, reference=dtrain, categorical_feature=cat_features)
                model = lgb.train(params, dtrain, valid_sets=[dval], callbacks=[lgb.early_stopping(50, verbose=False)])
                pred = model.predict(X_val)
            else:
                model = CatBoostRegressor(**params, cat_features=cat_features)
                model.fit(X_tr, y_tr, eval_set=(X_val, y_val), verbose=False)
                pred = model.predict(X_val)
                
            scores.append(np.sqrt(np.mean((pred - y_val)**2))) # RMSE
            
        return np.mean(scores)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=30) # 試行回数
    print(f"🏆 Best Params for {model_name}: {study.best_params}")
    return study.best_params

In [ ]:
# メイン処理
# 1. データの準備
print("データ前処理中...")
train_processed, features, target = preprocess_data(df, df_land, is_train=True)
train_processed = train_processed.dropna(subset=['unit_area'])
train_stats = {
    'unit_area': train_processed['unit_area'].median(),
    'year_built': train_processed['year_built'].median()
}
test_processed, _, _ = preprocess_data(test, df_land, is_train=False, fill_value=train_stats['unit_area'])
# 【Train独自の処理】
# 面積がないデータは教師として質が悪いので、行ごと消す



print(f"使用する特徴量: {len(features)}個")
print(features)
use_features = [f for f in features if f in train_processed.columns]
cat_features = [c for c in use_features if train_processed[c].dtype.name == 'category']
# ※CatBoost用にカテゴリ変数の名前リストも用意
cat_features_names = list(cat_features)
# 特徴量リスト（features）の緊急メンテナンス
# 特徴量リスト（features）のメンテナンス
# ---------------------------------------------------------

# 1. 絶対に使いたい数値データ
# (前の処理で作った station_minutes や log_land_price など)
num_targets = [
    'unit_area', 
    'log_land_price', 
    'dist_to_land_price', 
    'building_month', 
    'station_minutes'
]
# 実際にデータフレームにあるものだけ残す
final_num_features = [c for c in num_targets if c in train_processed.columns]

# 2. カテゴリ変数（LightGBM/CatBoostに任せるやつ）
cat_targets = [
    'addr1_2', 
    'rosen_name1', 
    'eki_name1', 
    'building_structure', 
    'snapshot_window_direction', 
    'madori_kind_all'
]
# 実際にあり、かつデータ型がちゃんと 'category' になっているものだけ残す
final_cat_features = [
    c for c in cat_targets 
    if c in train_processed.columns and train_processed[c].dtype.name == 'category'
]

# 3. ターゲットエンコーディング済みの列 (TE_ から始まる列)
# もし作っていれば自動で拾う
te_features = [c for c in train_processed.columns if c.startswith('TE_')]

# === 合体して features を上書き ===
features = final_num_features + final_cat_features + te_features

print(f"特徴量を {len(features)} 個に厳選しました！")
print("使う特徴量リスト:", features)
# ---------------------------------------------------------

# 1. 絶対に使いたい数値データ
# (前の処理で作った station_minutes や log_land_price など)
num_targets = [
    'unit_area', 
    'log_land_price', 
    'dist_to_land_price', 
    'building_month', 
    'station_minutes'
]
# 実際にデータフレームにあるものだけ残す
final_num_features = [c for c in num_targets if c in train_processed.columns]

# 2. カテゴリ変数（LightGBM/CatBoostに任せるやつ）
cat_targets = [
    'addr1_2', 
    'rosen_name1', 
    'eki_name1', 
    'building_structure', 
    'snapshot_window_direction', 
    'madori_kind_all'
]
# 実際にあり、かつデータ型がちゃんと 'category' になっているものだけ残す
final_cat_features = [
    c for c in cat_targets 
    if c in train_processed.columns and train_processed[c].dtype.name == 'category'
]

# 3. ターゲットエンコーディング済みの列 (TE_ から始まる列)
# もし作っていれば自動で拾う
te_features = [c for c in train_processed.columns if c.startswith('TE_')]

# === 合体して features を上書き ===
features = final_num_features + final_cat_features + te_features

print(f"特徴量を {len(features)} 個に厳選しました！")
print("使う特徴量リスト:", features)

# 2. Optunaが必要なら回す
if Config.run_optuna_lgb:
    best_p = run_optuna('LightGBM', train_processed, target, features, final_cat_features)
    Config.lgb_params.update(best_p) # 結果をConfigに上書き

if Config.run_optuna_cb:
    best_p = run_optuna('CatBoost', train_processed, target, features, cat_features_names)
    Config.cb_params.update(best_p)

# 3. LightGBM 学習
oof_lgb, pred_lgb, score_lgb = train_model(
    'LightGBM', train_processed, target, test_processed, 
    Config.lgb_params, features, cat_features
)

# 4. CatBoost 学習
oof_cb, pred_cb, score_cb = train_model(
    'CatBoost', train_processed, target, test_processed, 
    Config.cb_params, features, cat_features_names  
)

# 5. アンサンブル (加重平均)
# 単純平均(0.5ずつ).CVスコアが良い方を少し重くするのもアリ
weight_lgb = 0.5
weight_cb = 0.5

final_log_pred = (pred_lgb * weight_lgb) + (pred_cb * weight_cb)
final_oof = (oof_lgb * weight_lgb) + (oof_cb * weight_cb)

# 最終スコア確認
final_cv = mean_absolute_percentage_error(np.expm1(target), np.expm1(final_oof)) * 100

print("\n" + "="*40)
print(f"LightGBM CV: {score_lgb:.4f}%")
print(f"CatBoost CV: {score_cb:.4f}%")
print(f"Ensemble CV: {final_cv:.4f}%  <-- これが最終スコア！")
print("="*40)

# 6. CSV作成
submission = pd.DataFrame({'id': test['id'], 'prediction': np.expm1(final_log_pred) * test_processed['unit_area']})
submission.to_csv('submission_ensemble.csv', index=False, header=False)
print("submission_ensemble.csv Saved!")

データ前処理中...


使用する特徴量: 10個
['unit_area', 'log_land_price', 'dist_to_land_price', 'building_month', 'station_minutes', 'addr1_2', 'rosen_name1', 'eki_name1', 'building_structure', 'madori_kind_all']
特徴量を 10 個に厳選しました！
使う特徴量リスト: ['unit_area', 'log_land_price', 'dist_to_land_price', 'building_month', 'station_minutes', 'addr1_2', 'rosen_name1', 'eki_name1', 'building_structure', 'madori_kind_all']
特徴量を 10 個に厳選しました！
使う特徴量リスト: ['unit_area', 'log_land_price', 'dist_to_land_price', 'building_month', 'station_minutes', 'addr1_2', 'rosen_name1', 'eki_name1', 'building_structure', 'madori_kind_all']
🚀 LightGBM Training Started
  Fold 1 MAPE: 14.8569%
  Fold 2 MAPE: 14.9425%
  Fold 3 MAPE: 14.8103%
  Fold 4 MAPE: 14.9476%
  Fold 5 MAPE: 14.8400%
🚀 LightGBM Training Completed. Average MAPE: 14.8795%

🚀 CatBoost Training Started
  Fold 1 MAPE: 15.5928%
